## Basic script to load census tracks data provided by statistics Canada into PostGIS

With this script the data from the census tracks is taken and uploaed to the PostGIS database. 

### Steps that are taken
- Select the type of census data that will be uploaded to the database
- Merge the census track shapes with the census data by using the DGUID. The DGUID is the common identifier in both data sets
- Start a connection to the data base and upload the data
- Save the data as geojson

### Data source
The data is provided from the database, see how to set up a database and upload the data in this github. <br>
Following sources are used to create the data base.

#### Census data
- https://www12.statcan.gc.ca/census-recensement/2021/geo/sip-pis/boundary-limites/index2021-eng.cfm?year=21
- https://www150.statcan.gc.ca/n1/en/catalogue/98-401-X#wb-auto-2
- Source: Statistics Canada
- Publisher: Statistics Canada
- Accessed: 19.05.2024
- Date modified by publisher: 2023-09-29

#### Shape data for Montreal

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from census_select import select_data
from config import db_username, db_password, db_host, db_port, db_name

In [2]:
# Select the type of census data that will be uploaded to the database, uncomment the Name variable you want to select.
# See census_select.py file

# Name = 'census_population'
# Name = 'census_private_dwellings'
# Name = 'census_total_income_2020'
# Name = 'census_total_income_2019'
# Name = 'census_household_income_2020'
# Name = 'census_household_type'
# Name = 'census_household_income_after_tax_2020'
# Name = 'census_LIM_2020'
# Name = 'census_highest_certificate_15years'
# Name = 'census_highest_certificate_25_64years'
# Name = 'census_employ'
# Name = 'census_shelter_cost'
Name = 'census_citizenship'
# Name = 'census_religion'
# Name ='census_minority'
# Name = 'census_housing_situation'
# Name = 'cencus_immigtant_status'

In [ ]:
# Define the folder path (you can use an absolute or relative path)
folder_path = "/home/jovyan/work/Data/"

# Set the environment variable, for example 'MY_FOLDER'
os.environ["Data_folder"] = folder_path

# You can verify it's set by printing it
print("Environment variable Data_folder is set to:", os.environ["Data_folder"])

In [ ]:
# Get the folder path from the environment variable
folder_path = os.environ.get("Data_folder")
if not folder_path:
    raise ValueError("The environment variable Data_folder is not set.")

In [ ]:
# Import the island boundary shapefile, it can be also a complete city or region
file_path = os.path.join(folder_path, "Administrative_boundaries/Montreal_island.geojson")
Island_boundary = gpd.read_file(file_path)
Island_boundary.head(5)

In [3]:
# # Import the island boundary shapefile, it can be also a complete city or region
# Island_boundary = gpd.read_file('Borough_boundary/Montreal_island.geojson')
# Island_boundary.head(3)

,geometry
0,"POLYGON ((-73.83182 45.39035, -73.83687 45.392..."


In [4]:
# Import the complete census tracks for Canada as provided by statistics Canada
file_path = os.path.join(folder_path, "Census/lct_000b21a_e.shp")
Census_tracks = gpd.read_file(file_path)
Census_tracks.head(1)

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,geometry
0,5370001.08,2021S05075370001.08,0001.08,1.6383,35,"POLYGON ((7196507.366 869787.991, 7196501.617 ..."


In [5]:
# Project the cencus tracks to crs 4326
Census_tracks_projected = Census_tracks.to_crs(epsg=4326)
# Census_tracks_projected.head(3)

In [6]:
# Project the island boundary to crs 4326
Island_boundary_projected = Island_boundary.to_crs(epsg=4326)
# Island_boundary_projected.head(3)

In [7]:
# Clipp the cencus tracks from the big file to focus area
Census_tracks_boundary_clipp = gpd.overlay(Census_tracks_projected, Island_boundary_projected, how='intersection')
Census_tracks_boundary_clipp

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,geometry
0,4620001.00,2021S05074620001.00,0001.00,0.4643,24,"POLYGON ((-73.50741 45.61055, -73.50737 45.610..."
1,4620002.00,2021S05074620002.00,0002.00,0.3858,24,"POLYGON ((-73.51736 45.60154, -73.51748 45.601..."
2,4620003.00,2021S05074620003.00,0003.00,0.7401,24,"POLYGON ((-73.52152 45.60276, -73.52198 45.602..."
3,4620004.00,2021S05074620004.00,0004.00,0.4470,24,"POLYGON ((-73.51915 45.59794, -73.51927 45.597..."
4,4620005.00,2021S05074620005.00,0005.00,0.5652,24,"POLYGON ((-73.51332 45.60286, -73.51316 45.602..."
...,...,...,...,...,...,...
537,4620194.02,2021S05074620194.02,0194.02,0.5939,24,"POLYGON ((-73.52792 45.60827, -73.52823 45.607..."
538,4620421.03,2021S05074620421.03,0421.03,0.1887,24,"POLYGON ((-73.66913 45.52936, -73.66892 45.529..."
539,4620421.04,2021S05074620421.04,0421.04,0.3673,24,"POLYGON ((-73.67149 45.52456, -73.6715 45.5245..."
540,4620421.05,2021S05074620421.05,0421.05,0.3041,24,"POLYGON ((-73.66523 45.53069, -73.66538 45.530..."


In [8]:
# Save census tracks as geojson into the output folder
Census_tracks_boundary_clipp.to_file('output/census_tracks.geojson', driver='GeoJSON')

In [9]:
# Load the csv file with the census data, this file includes all the data for Canada
file_path = os.path.join(folder_path, "Census/98-401-X2021007_English_CSV_data.csv")
Census_data = pd.read_csv(file_path, encoding='latin1')
# Display the first few rows of the DataFrame
Census_data.head(3)

C:\Users\ReinerBraun\AppData\Local\Temp\ipykernel_12936\1186300541.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  Census_data = pd.read_csv('Census_data/98-401-X2021007_English_CSV_data.csv', encoding='latin1')


,CENSUS_YEAR,DGUID,ALT_GEO_CODE,GEO_LEVEL,GEO_NAME,TNR_SF,TNR_LF,DATA_QUALITY_FLAG,CHARACTERISTIC_ID,CHARACTERISTIC_NAME,...,C2_COUNT_MEN+,SYMBOL.1,C3_COUNT_WOMEN+,SYMBOL.2,C10_RATE_TOTAL,SYMBOL.3,C11_RATE_MEN+,SYMBOL.4,C12_RATE_WOMEN+,SYMBOL.5
0,2021,2021S0503932,932.0,Census metropolitan area,Abbotsford - Mission,2.7,3.7,0,1,"Population, 2021",...,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...
1,2021,2021S0503932,932.0,Census metropolitan area,Abbotsford - Mission,2.7,3.7,0,2,"Population, 2016",...,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...
2,2021,2021S0503932,932.0,Census metropolitan area,Abbotsford - Mission,2.7,3.7,0,3,"Population percentage change, 2016 to 2021",...,NaN,...,NaN,...,8.4,NaN,NaN,...,NaN,...


In [10]:
# Merge country_shapes to country names on NAME Column. 
Census_data_merged = pd.merge(Census_tracks_boundary_clipp, Census_data, on='DGUID', how='inner')
Census_data_merged.head(5)

,CTUID,DGUID,CTNAME,LANDAREA,PRUID,geometry,CENSUS_YEAR,ALT_GEO_CODE,GEO_LEVEL,GEO_NAME,...,C2_COUNT_MEN+,SYMBOL.1,C3_COUNT_WOMEN+,SYMBOL.2,C10_RATE_TOTAL,SYMBOL.3,C11_RATE_MEN+,SYMBOL.4,C12_RATE_WOMEN+,SYMBOL.5
0,4620001.00,2021S05074620001.00,0001.00,0.4643,24,"POLYGON ((-73.50741 45.61055, -73.50737 45.610...",2021,4620001.0,Census tract,4620001.00,...,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...
1,4620001.00,2021S05074620001.00,0001.00,0.4643,24,"POLYGON ((-73.50741 45.61055, -73.50737 45.610...",2021,4620001.0,Census tract,4620001.00,...,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...
2,4620001.00,2021S05074620001.00,0001.00,0.4643,24,"POLYGON ((-73.50741 45.61055, -73.50737 45.610...",2021,4620001.0,Census tract,4620001.00,...,NaN,...,NaN,...,1.7,NaN,NaN,...,NaN,...
3,4620001.00,2021S05074620001.00,0001.00,0.4643,24,"POLYGON ((-73.50741 45.61055, -73.50737 45.610...",2021,4620001.0,Census tract,4620001.00,...,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...
4,4620001.00,2021S05074620001.00,0001.00,0.4643,24,"POLYGON ((-73.50741 45.61055, -73.50737 45.610...",2021,4620001.0,Census tract,4620001.00,...,NaN,...,NaN,...,NaN,...,NaN,...,NaN,...


In [11]:
# Select the data of interest from the census data, see the Name selection file 
Census_data_upload_data = select_data(Name, Census_data_merged)
# Census_data_upload_data.head(3)

In [12]:
# Creates the connection to the POSGIS data base
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

In [13]:
# Upload the Census_tracks_boundary_clipp data from the GeoDataFrame to PostgreSQL/PostGIS
# Uncomment if you want to upload, a running PostGIS is needed

#Census_tracks_boundary_clipp.to_postgis("census_tracks", engine, if_exists='replace')

In [14]:
# Upload the data from the GeoDataFrame to PostgreSQL/PostGIS
# Uncomment if you want to upload, a running PostGIS is needed

#Census_data_upload_data.to_postgis(Name, engine, if_exists='replace')

In [15]:
# Save the type of census data to a geojson
Census_data_upload_data.to_file('output/'+ Name +'.geojson', driver='GeoJSON')